# testing ANPE deployment on NSA observations

In [1]:
import os, sys
import numpy as np
from scipy import stats
from sedflow import obs as Obs
from sedflow import train as Train

# torch
import torch
from sbi import utils as Ut
from sbi import inference as Inference

In [2]:
y_nsa = Obs.load_nsa_data(test_set=False)

/home/chhahn/projects/SEDflow/src/sedflow/train.py:106: RuntimeWarning: invalid value encountered in log10
  return 22.5 - 2.5 * np.log10(flux)


In [3]:
####################################################################
# set prior (this is fixed)
####################################################################
prior_low = [7, 0., 0., 0., 0., 1e-2, np.log10(4.5e-5), np.log10(4.5e-5), 0, 0., -2.]
prior_high = [12.5, 1., 1., 1., 1., 13.27, np.log10(1.5e-2), np.log10(1.5e-2), 3., 3., 1.]
lower_bounds = torch.tensor(prior_low)
upper_bounds = torch.tensor(prior_high)
prior = Ut.BoxUniform(low=lower_bounds, high=upper_bounds, device='cpu')

In [4]:
####################################################################
# load trained ANPE
####################################################################
sample = 'toy'
nhidden = 500
nblocks = 15
itrain = 2
####################################################################

x_test, y_test = Train.load_data('test', version=1, sample=sample, params='thetas_unt')
x_test[:,6] = np.log10(x_test[:,6])
x_test[:,7] = np.log10(x_test[:,7])

fanpe = os.path.join(Train.data_dir(), 'anpe_thetaunt_magsigz.%s.%ix%i.%i.pt' % (sample, nhidden, nblocks, itrain))

anpe = Inference.SNPE(prior=prior,
                      density_estimator=Ut.posterior_nn('maf', hidden_features=nhidden, num_transforms=nblocks),
                      device='cpu')
anpe.append_simulations(
    torch.as_tensor(x_test.astype(np.float32)),
    torch.as_tensor(y_test.astype(np.float32)))

p_x_y_estimator = anpe._build_neural_net(torch.as_tensor(x_test.astype(np.float32)), torch.as_tensor(y_test.astype(np.float32)))
p_x_y_estimator.load_state_dict(torch.load(fanpe))

anpe._x_shape = Ut.x_shape_from_simulation(torch.as_tensor(y_test.astype(np.float32)))

hatp_x_y = anpe.build_posterior(p_x_y_estimator)

In [5]:
print('%i hyperparameters' % np.sum(p.numel() for p in p_x_y_estimator.parameters() if p.requires_grad))

7890330 hyperparameters


/home/chhahn/.conda/envs/torch-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  """Entry point for launching an IPython kernel.


In [6]:
def get_posterior(y_nsa_i, nmcmc=10000, **kwargs):
    ''' given [mag, uncertainty, redshift] of a galaxy, draw nmcmc samples of
    the posterior.
    '''
    mcmc_anpe = hatp_x_y.sample((nmcmc,), x=torch.as_tensor(y_nsa_i), show_progress_bars=True, **kwargs)
    return np.array(mcmc_anpe)

In [8]:
mcmcs = []
for igal in [6005]: #range(6000,6100): 
    print(y_nsa[igal])
    _mcmc_i = get_posterior(y_nsa[igal])
    mcmcs.append(_mcmc_i)

[18.714787   18.016953   17.825827   17.731255   17.677258    0.08050386
  0.02300007  0.02354047  0.17673106  0.06297234  0.02071856]


Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

                        constant for `log_prob()`. However, only
                        1% posterior samples are within the
                        prior support. It may take a long time to collect the
                        remaining 9945 samples.
                        Consider interrupting (Ctrl-C) and either basing the
                        estimate of the normalizing constant on fewer samples (by
                        calling `posterior.leakage_correction(x_o,
                        num_rejection_samples=N)`, where `N` is the number of
                        samples you want to base the
                        estimate on (default N=10000), or not estimating the
                        normalizing constant at all
                        (`log_prob(..., norm_posterior=False)`. The latter will
                        result in an unnormalized `log_prob()`.


KeyboardInterrupt: 

some posteriors take a very long time to sample. Lets see what's up with that.

In [20]:
igal = 6005
print(y_nsa[igal])
mcmc_i = np.array(hatp_x_y.sample((10000,), x=torch.as_tensor(y_nsa[igal]), show_progress_bars=True))

[18.714787   18.016953   17.825827   17.731255   17.677258    0.08050386
  0.02300007  0.02354047  0.17673106  0.06297234  0.02071856]


Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

                        constant for `log_prob()`. However, only
                        0% posterior samples are within the
                        prior support. It may take a long time to collect the
                        remaining 9958 samples.
                        Consider interrupting (Ctrl-C) and either basing the
                        estimate of the normalizing constant on fewer samples (by
                        calling `posterior.leakage_correction(x_o,
                        num_rejection_samples=N)`, where `N` is the number of
                        samples you want to base the
                        estimate on (default N=10000), or not estimating the
                        normalizing constant at all
                        (`log_prob(..., norm_posterior=False)`. The latter will
                        result in an unnormalized `log_prob()`.


KeyboardInterrupt: 

In [8]:
import signal

igal = 6005
class TimeoutException(Exception):   # Custom exception class
    pass

def timeout_handler(signum, frame):   # Custom signal handler
    raise TimeoutException

# Change the behavior of SIGALRM
signal.signal(signal.SIGALRM, timeout_handler)

for i in range(3):
    # Start the timer. Once 5 seconds are over, a SIGALRM signal is sent.
    signal.alarm(300)    
    # This try/except loop ensures that 
    #   you'll catch TimeoutException when it's sent.
    try:
        mcmc_i = np.array(hatp_x_y.sample((10000,), x=torch.as_tensor(y_nsa[igal]), show_progress_bars=True))
    except TimeoutException:
        print("timed out")
        continue # continue the for loop if function A takes more than 5 second
    else:
        # Reset the alarm
        signal.alarm(0)

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

                        constant for `log_prob()`. However, only
                        1% posterior samples are within the
                        prior support. It may take a long time to collect the
                        remaining 9947 samples.
                        Consider interrupting (Ctrl-C) and either basing the
                        estimate of the normalizing constant on fewer samples (by
                        calling `posterior.leakage_correction(x_o,
                        num_rejection_samples=N)`, where `N` is the number of
                        samples you want to base the
                        estimate on (default N=10000), or not estimating the
                        normalizing constant at all
                        (`log_prob(..., norm_posterior=False)`. The latter will
                        result in an unnormalized `log_prob()`.


timed out


Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

                        constant for `log_prob()`. However, only
                        0% posterior samples are within the
                        prior support. It may take a long time to collect the
                        remaining 9958 samples.
                        Consider interrupting (Ctrl-C) and either basing the
                        estimate of the normalizing constant on fewer samples (by
                        calling `posterior.leakage_correction(x_o,
                        num_rejection_samples=N)`, where `N` is the number of
                        samples you want to base the
                        estimate on (default N=10000), or not estimating the
                        normalizing constant at all
                        (`log_prob(..., norm_posterior=False)`. The latter will
                        result in an unnormalized `log_prob()`.


KeyboardInterrupt: 